## Custom Agent to Output Tx Params using Tool


In [2]:
from graph.converter_tool import generate_tx_params

from_address = "0xd8da6bf26964af9d7eed9e03e53415d37aa96045"

In [3]:
description = "Send 0.01 ETH to the address 0x8c575b178927fF9A70804B8b4F7622F7666bB360."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0x8c575b178927fF9A70804B8b4F7622F7666bB360".lower()
assert res.data == "0x"
assert res.value == "0x2386f26fc10000"

From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0x8c575b178927fF9A70804B8b4F7622F7666bB360
Data: 0x
Value: 0x2386f26fc10000


In [4]:
description = "Transfer 100 USDT to Scott's Ethereum address."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xdAC17F958D2ee523a2206206994597C13D831ec7".lower()
data = "0xa9059cbb000000000000000000000000b859d7eb5e319e54dc23c83dfe93e649b88202b60000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data == data
assert res.value == "0x0"

Fetching ABI from Etherscan for address: 0xdac17f958d2ee523a2206206994597c13d831ec7
Not a proxy contract, returning the ABI
From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0xdAC17F958D2ee523a2206206994597C13D831ec7
Data: 0xa9059cbb000000000000000000000000b859d7eb5e319e54dc23c83dfe93e649b88202b60000000000000000000000000000000000000000000000000000000005f5e100
Value: 0x0


In [5]:
description = "Approve 100 USDT for swapping on a decentralized exchange."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xdAC17F958D2ee523a2206206994597C13D831ec7".lower()
data = "0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data == data
assert res.value == "0x0"

Not a proxy contract, returning the ABI
From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0xdAC17F958D2ee523a2206206994597C13D831ec7
Data: 0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000005f5e100
Value: 0x0


In [6]:
description = "Swap 100 USDT to USDC on the chosen decentralized exchange."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D".lower()
# Data is omitted since deadline is not deterministic
assert res.value == "0x0"

Fetching ABI from Etherscan for address: 0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D
Not a proxy contract, returning the ABI
From: 0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045
To: 0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D
Data: 0x38ed17390000000000000000000000000000000000000000000000000000000005f5e100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000d8da6bf26964af9d7eed9e03e53415d37aa960450000000000000000000000000000000000000000000000000000000066e692510000000000000000000000000000000000000000000000000000000000000002000000000000000000000000dac17f958d2ee523a2206206994597c13d831ec7000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
Value: 0x0


In [7]:
description = "Stake 0.03 ETH to Lido to receive stETH."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84".lower()
data = "0xa1903eab0000000000000000000000000000000000000000000000000000000000000000"
assert res.data == data
assert res.value == "0x6a94d74f430000"

Fetching ABI from Etherscan for address: 0xae7ab96520de3a18e5e111b5eaab095312d7fe84
Proxy contract detected at 0xae7ab96520de3a18e5e111b5eaab095312d7fe84
Implementation address: 0x17144556fd3424EDC8Fc8A4C940B2D04936d17eb
Fetching ABI from Etherscan for address: 0x17144556fd3424EDC8Fc8A4C940B2D04936d17eb
Not a proxy contract, returning the ABI
From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0xae7ab96520de3a18e5e111b5eaab095312d7fe84
Data: 0xa1903eab0000000000000000000000000000000000000000000000000000000000000000
Value: 0x6a94d74f430000


## Custom Agent to Output Tx Params using LLM


In [8]:
from graph.converter_llm import generate_tx_params

from_address = "0xd8da6bf26964af9d7eed9e03e53415d37aa96045"

In [9]:
description = "Send 0.01 ETH to the address 0x8c575b178927fF9A70804B8b4F7622F7666bB360."
res = await generate_tx_params(description, from_address)
print(res.json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0x8c575b178927fF9A70804B8b4F7622F7666bB360".lower()
assert res.data == "0x"
assert res.value == "0x2386f26fc10000"

{"from_address": "0xd8da6bf26964af9d7eed9e03e53415d37aa96045", "to_address": "0x8c575b178927fF9A70804B8b4F7622F7666bB360", "data": "0x", "value": "0x2386f26fc10000"}


In [10]:
description = "Transfer 100 USDT to Scott's Ethereum address."
res = await generate_tx_params(description, from_address)
print(res.json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xdAC17F958D2ee523a2206206994597C13D831ec7".lower()
data = "0xa9059cbb000000000000000000000000b859d7eb5e319e54dc23c83dfe93e649b88202b60000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data == data
assert res.value == "0x0"

Fetching ABI from Etherscan for address: 0xdAC17F958D2ee523a2206206994597C13D831ec7
Not a proxy contract, returning the ABI
{"from_address": "0xd8da6bf26964af9d7eed9e03e53415d37aa96045", "to_address": "0xdAC17F958D2ee523a2206206994597C13D831ec7", "data": "0xa9059cbb000000000000000000000000b859d7eb5e319e54dc23c83dfe93e649b88202b60000000000000000000000000000000000000000000000000000000005f5e100", "value": "0x0"}


In [11]:
description = "Approve 100 USDT for swapping on a decentralized exchange."
res = await generate_tx_params(description, from_address)
print(res.json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xdAC17F958D2ee523a2206206994597C13D831ec7".lower()
data = "0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data == data
assert res.value == "0x0"

{"from_address": "0xd8da6bf26964af9d7eed9e03e53415d37aa96045", "to_address": "0xdAC17F958D2ee523a2206206994597C13D831ec7", "data": "0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000005f5e100", "value": "0x0"}


In [13]:
description = "Swap 100 USDT to USDC on the chosen decentralized exchange."
res = await generate_tx_params(description, from_address)
print(res.json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D".lower()
# Data is omitted since deadline is not deterministic
assert res.value == "0x0"

{"from_address": "0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045", "to_address": "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", "data": "0x38ed17390000000000000000000000000000000000000000000000000000000005f5e100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000d8da6bf26964af9d7eed9e03e53415d37aa960450000000000000000000000000000000000000000000000000000000066e685940000000000000000000000000000000000000000000000000000000000000002000000000000000000000000dac17f958d2ee523a2206206994597c13d831ec7000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "value": "0x0"}


In [14]:
description = "Stake 0.03 ETH to Lido to receive stETH."
res = await generate_tx_params(description, from_address)
print(res.json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84".lower()
data = "0xa1903eab0000000000000000000000000000000000000000000000000000000000000000"
assert res.data == data
assert res.value == "0x6a94d74f430000"

{"from_address": "0xd8da6bf26964af9d7eed9e03e53415d37aa96045", "to_address": "0xae7ab96520de3a18e5e111b5eaab095312d7fe84", "data": "0xa1903eab0000000000000000000000000000000000000000000000000000000000000000", "value": "0x6a94d74f430000"}
